In [ ]:
!nvidia-smi

Sun Aug 28 10:13:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets transformers

In [ ]:
import re
from tqdm import tqdm
from tqdm.auto import trange
import time
import os

from typing import List, Dict, Union, Tuple, NoReturn

import pandas as pd
from pandas.core.series import Series
import json

import numpy as np
import string

import nltk    
from nltk import tokenize    
nltk.download('punkt')   

from gensim.models import KeyedVectors

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

import keras
from keras.models import load_model, model_from_json, Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Flatten, LSTM, Bidirectional, Dropout, concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import Input, Model
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import average

import transformers
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import BertTokenizerFast, BatchEncoding, PreTrainedTokenizerFast, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_metric

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import gc


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
GDRIVE_PATH:str = '/content/drive/MyDrive'
DATASET_ROOT_PATH: str = os.path.join(GDRIVE_PATH, 'dataset', 'projeto-final')
DATASET_IMPRENSA: str = os.path.join(DATASET_ROOT_PATH, 'imprensa')
DATASET_MENCOES: str = os.path.join(DATASET_ROOT_PATH, 'mencoes')
CONTEUDO_INDISPONIVEL: str = 'CONTEUDO_INDISPONIVEL'
STOP_WORDS_FILE: str = os.path.join(DATASET_ROOT_PATH, 'stop-words.txt')

EMBEDDING_GLOVE_50 : str = os.path.join(GDRIVE_PATH, 'model', 'embedding','glove_s50.txt')
EMBEDDING_DIM_SIZE: int = 50

LABEL_NAMES: List[str] = ['Negativa', 'Neutra', 'Positiva']
LABEL_DICT_CONV_MENCOES : Dict[int,str] = {-5:'Negativa', 0:'Neutra', 5:'Positiva'}

MAX_SENTENCE_LENGTH: int = 1000 
MODEL_PATH: str = os.path.join(GDRIVE_PATH, 'model', 'projeto-final')

# palavras comumente incorretas
SUBSTITUICOES_COMUNS_PALAVRAS_INCORRETAS: Dict[str,str] = {
    'covid-00': 'coronavírus',  'privatizacao':  'privatização', 'leilao':  'leilão',
    'inflacao':  'inflação', 'bilhao':  'bilhão', 'concessoes': 'concessões',
    'aprovacao': 'aprovação', 'covid': 'coronavírus', 'desestatizacao': 'desestatização',
    'governanca': 'governança', 'atuacao': 'atuação', 'emissoes': 'emissões', 
    'manutencao': 'manutenção', 'licitacao': 'licitação', 'protecao': 'proteção',
    'emissao': 'emissão', 'contratacao': 'contratação', 'aquisicao': 'aquisição',
    'arrecadacao': 'arrecadação', 'votacao': 'votação', 'ampliacao': 'ampliação',
    'negociacao': 'negociação', 'vacinacao': 'vacinação', 'inadimplencia': 'inadimplência', 
    'poupanca': 'poupança', 'realizacao': 'realização', 'suspensao': 'suspensão', 
    'preservacao': 'preservação', 'estruturacao': 'estruturação', 'fiscalizacao': 'fiscalização',
    'capitalizacao': 'capitalização', 'conservacao': 'conservação', 'prestacao': 'prestação',
    'cobranca': 'cobrança', 'transicao': 'transição', 'remuneracao': 'remuneração',
    'liberacao': 'liberação', 'discussoes': 'discussões', 'universalizacao': 'universalização',
    'aculpa': 'culpa', 'deverao': 'deverão', 'elaboracao': 'elaboração',
    'contribuicao': 'contribuição', 'mineracao': 'mineração', 'adesao': 'adesão',
    'modernizacao': 'modernização','regulacao': 'regulação', 'projecao': 'projeção',
    'regulatorio': 'regulatório', 'centrao': 'centrão', 'avancos': 'avanços',
    'climatica': 'climática', 'variacao': 'variação', 'implantacao': 'implantação',
    'implementacao': 'implementação', 'projecoes': 'projeções', 'senadorhumberto': 'senador',
    'trilhao': 'trilhão', 'reeleicao':  'reeleição', 'restricoes': 'restrições',
    'elevacao': 'elevação', 'percepcao': 'percepção', 'importacao': 'importação',
    'exportacao':  'exportação', 'valorizacao':  'valorização', 'licitacoes': 'licitações', 
    'adocao': 'adoção', 'trilhoes': 'trilhões', 'sustentaveis': 'sustentáveis',
    'aviacao': 'aviação', 'pregao': 'pregão', 'recessao': 'recessão', 'reacao': 'reação',
    'mineconomia' : 'ministro', 'regulamentacao' : 'regulamentação', 'movimentacao' : 'movimentação',
    'p/': 'para'
}

MODEL_TRAINED_PATH: str = os.path.join(MODEL_PATH, 'bert')
MODEL_TRAINED_LOG: str = os.path.join(MODEL_TRAINED_PATH, 'trainer.log')

BATCH_SIZE = 16
MAX_LEN = 512

BASE_BERT_MODEL: str = 'neuralmind/bert-base-portuguese-cased'

In [ ]:
# Liberar e monitorar memória da GPU
def destroy_model(model):
    del model
    gc.collect()
    torch.cuda.empty_cache()

def destroy_tokenizer(tokenizer):
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()

def get_gpu_memory_status():
    total = (torch.cuda.get_device_properties(0).total_memory)/(1024 **2)
    reserved = (torch.cuda.memory_reserved(0))/(1024 **2)
    allocated = (torch.cuda.memory_allocated(0))/(1024 **2)
    return f"Total: {total:.2f} | Reserved: {reserved:.2f} | Allocated: {allocated:.2f}"

In [ ]:
def read_dataset_imprensa() -> Tuple[np.ndarray, np.ndarray] :
    '''
        Carrega o dataset imprensa retornando dois numpy`s, o primeiro sáo os textos e o segundo os labels
    '''
    lista_texto : List[str] = []
    lista_avaliacao: List[str] = []

    for arq in os.listdir(DATASET_IMPRENSA):
        if not '.csv' in arq:
            continue
        df = pd.read_csv(os.path.join(DATASET_IMPRENSA, arq), sep='|')
        df = df[df["texto_artigo"] != 'CONTEUDO_INDISPONIVEL']
        df_texto = df[['texto_artigo']]
        df_avaliacao = df[['Avaliação']]
        
        lista_texto_aux = df_texto.astype(str).values.tolist()
        lista_avaliacao_aux = df_avaliacao.astype(str).values.tolist()
        
        for (texto,label) in zip(lista_texto_aux, lista_avaliacao_aux):
            lista_texto.append(texto[0])
            lista_avaliacao.append(label[0])

    return (lista_texto, lista_avaliacao)

In [ ]:
def read_dataset_mencoes(debug: bool = False) -> Tuple[np.ndarray, np.ndarray] :
    '''
        Carrega o dataset mencoes retornando dois numpy`s, o primeiro sáo os textos e o segundo os labels
    '''
    lista_texto : List[str] = []
    lista_avaliacao: List[str] = []
    qtd_vazias: int = 0

    for dir_ano in os.listdir(DATASET_MENCOES):
        full_path: str = os.path.join(DATASET_MENCOES, dir_ano)
        for arq in os.listdir(full_path):
            if not '.csv' in arq:
                continue
            df = pd.read_csv(os.path.join(full_path, arq), sep='|')
            # remover linhas vazias
            df = df.dropna(subset=['content'])
            df = df.drop_duplicates(subset=['content'])
            
            df_texto = df[['content']]
            df_avaliacao = df[['sentiment']]
            
            lista_texto_aux = df_texto.astype(str).values.tolist()
            lista_avaliacao_aux = df_avaliacao.astype(str).values.tolist()
            
            for (texto,label) in zip(lista_texto_aux, lista_avaliacao_aux):
                if len(texto[0].strip()) > 0 or texto[0] == 'nan':
                    #print(texto[0])
                    lista_texto.append(texto[0])
                    lista_avaliacao.append(LABEL_DICT_CONV_MENCOES[int(float(label[0]))])                
    return (lista_texto, lista_avaliacao)

In [ ]:
def load_stopwords() -> List[str]:
    """
    This function loads a stopword list from the *path* file and returns a 
    set of words. Lines begining by '#' are ignored.
    """

    # Set of stopwords
    stopwords = set([])

    # For each line in the file
    with open(STOP_WORDS_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if not re.search('^#', line) and len(line.strip()) > 0:
                stopwords.add(line.strip().lower())

    # inclusão dos tokens gerados incorretamente pelo word tokenize
    stopwords.add("``")
    stopwords.add("''")
    # Return the set of stopwords
    return stopwords

In [ ]:
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+[:]*)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+[:]*)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def trata_bndes(tweet):
    """Trata o nome do BNDES, que aparece muito e não esá"""
    #tweet = re.sub('BNDES', 'banco', tweet)  
    #tweet = re.sub('BNDS', 'banco', tweet)  
    
    return tweet

def trata_erros_escrita_comuns(tweet):
    for key in SUBSTITUICOES_COMUNS_PALAVRAS_INCORRETAS:
        tweet = re.sub(key, SUBSTITUICOES_COMUNS_PALAVRAS_INCORRETAS[key], tweet)
    return tweet

def pre_processar(text: str) -> str:
    text = remove_links(text)
    text = remove_users(text)
    text = remove_hashtags(text)
    text = remove_av(text)
    text = trata_bndes(text)
    text = trata_erros_escrita_comuns(text)

    return text.strip()

In [ ]:
def carrega_texto_treinamento() -> Tuple[List[str], List[str]]:
    '''
        Carrega todos os textos necessários para treinamento

        Returns:
            List[str] - Textos
            List[str] - Labels
    '''

    stop_words = load_stopwords()
    
    lista_texto_imprensa: List[str]
    lista_label_imprensa: List[str] 
    lista_texto_mencoes: List[str]
    lista_label_mencoes: List[str]
    lista_texto_completa: List[str] = []
    lista_texto_completa_final: List[str] = []
    lista_label_completa: List[str] = []


    (lista_texto_imprensa, lista_label_imprensa) = read_dataset_imprensa()
    (lista_texto_mencoes, lista_label_mencoes) = read_dataset_mencoes()

    lista_texto_completa.extend(lista_texto_imprensa)
    lista_texto_completa.extend(lista_texto_mencoes)
    lista_label_completa.extend(lista_label_imprensa)
    lista_label_completa.extend(lista_label_mencoes)


    for texto in tqdm(lista_texto_completa,'Limpando textos para entrada no modelo....'):
        lista_texto_completa_final.append(pre_processar(texto))

    return (lista_texto_completa_final, lista_label_completa)


In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='macro')["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
(lista_texto_completa,lista_label_completa) = carrega_texto_treinamento()
lista_label_conv: List[int] = np.array([LABEL_NAMES.index(label) for label in lista_label_completa])
print(f'Texto {len(lista_texto_completa)} Label {len(lista_label_conv)}')


Limpando textos para entrada no modelo....: 100%|██████████| 45440/45440 [00:10<00:00, 4263.51it/s]

Texto 45440 Label 45440


In [ ]:
lista_texto_completa[10011]

'“O PT sob o comando do Lula deixou o rombo de 42 bilhões na Petrobras;5 bilhões no fundo dos Correios, 20 bilhões no fundo da Petrobras; 13 bilhões no fundo do BB; 12 bilhões no fundo da Caixa, e 500 bilhões do BNDES para ditaduras. Nossa 4ª geração pagará pelos estragos do PT.”'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(BASE_BERT_MODEL, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(BASE_BERT_MODEL, do_lower_case=False)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [ ]:
model.config.hidden_size

768

In [ ]:
tokens = tokenizer.tokenize(' O bndes foi ')
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {lista_texto_completa[0]}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: Apos o governo ensaiar um recuo na proposta de financiar o Renda Cidada com dinheiro dos precatorios, o dia de ontem terminou com uma indefinicao em torno de como se pretende custear o programa social bolsonarista. Momentos depois de o ministro da Economia, Paulo Guedes, ter defendido um a fonte de recursos "saudavel, limpa e permanente" para o programa, o senador Marcio Bittar (MDB-AC), relator das PECs do Pacto Federativo e do auxilio emergencial, disse ao Valor que a ideia de usar os precatorios para essa finalidade estara implicita no texto, que ele esta finalizando. Ontem, enquanto membros do governo propagavam a versao de que Guedes foi "atropelado" nas discussões, Bittar afirmava que a ideia teve aval do presidente Jair Bolsonaro e do ministro. "Um projeto dessa magnitude jamais seria apresentado se nao tivesse o conhecimento e a aprovação do presidente da nacao e o carimbo de 'ok' do ministro da Economia", afirmou. Ontem pela manha, um ministro disse ao Valor que a i

In [ ]:
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)

[SEP] 102
[CLS] 101
[PAD] 0
[UNK] 100


In [ ]:
class BNDESSentimentDataset(torch.utils.data.Dataset):

  def __init__(self, textos, sentimentos, tokenizer, max_len):
    self.lista_texto = textos
    self.lista_sentimento = sentimentos
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.lista_texto)
  
  def __getitem__(self, item):
    texto = str(self.lista_texto[item])
    sentimento = self.lista_sentimento[item]

    encoding = self.tokenizer.encode_plus(
      texto,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'texto': texto,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'labels': torch.tensor(sentimento, dtype=torch.long)
    }

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(lista_texto_completa, lista_label_conv,
                                                    test_size=0.1,
                                                    random_state=0,
                                                    stratify=lista_label_conv)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.1,
                                                    random_state=0,
                                                    stratify=y_train)


train_dataset = BNDESSentimentDataset(textos=X_train,sentimentos=y_train,tokenizer=tokenizer,max_len=MAX_LEN)
val_dataset = BNDESSentimentDataset(textos=X_val,sentimentos=y_val,tokenizer=tokenizer,max_len=MAX_LEN)
test_dataset = BNDESSentimentDataset(textos=X_test,sentimentos=y_test,tokenizer=tokenizer,max_len=MAX_LEN)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir= MODEL_TRAINED_LOG,            # directory for storing logs
    logging_steps=500,
    evaluation_strategy='steps',
    save_strategy='steps',
    load_best_model_at_end=True

)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset ,            # evaluation dataset
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(MODEL_TRAINED_PATH)
destroy_model(model)
destroy_tokenizer(tokenizer)

Saving model checkpoint to /content/drive/MyDrive/model/projeto-final/bert
Configuration saved in /content/drive/MyDrive/model/projeto-final/bert/config.json
Model weights saved in /content/drive/MyDrive/model/projeto-final/bert/pytorch_model.bin


## Teste do Modelo

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_TRAINED_PATH, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(BASE_BERT_MODEL, do_lower_case=False)
model.to('cuda')

In [ ]:
loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
loop = tqdm(loader, leave=True)
final_output_loss = []
final_output_logits = []

for it, batch in enumerate(loop):

    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    outputs = model(input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels)
    
    final_output_loss.append(outputs.loss.detach().to('cpu').numpy())
    final_output_logits.append(outputs.logits.detach().to('cpu').numpy())

    input_ids.detach()
    attention_mask.detach()
    labels.detach
    labels = None
    input_ids = None
    attention_mask = None
    outputs.logits = None
    outputs.loss = None
    outputs = None
    gc.collect()
    torch.cuda.empty_cache()
    

100%|██████████| 284/284 [03:09<00:00,  1.50it/s]


In [ ]:
destroy_model(model)
get_gpu_memory_status()

'Total: 16280.88 | Reserved: 2244.00 | Allocated: 2084.78'

In [ ]:
test_preds = np.vstack(final_output_logits)
test_preds = np.argmax(test_preds, axis=-1)

In [ ]:
y_true = np.array(y_test).ravel()
y_pred = test_preds.ravel()

score = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
print('F1 Score (Macro) - ', score)
score = accuracy_score(y_true=y_true, y_pred=y_pred)
print('Acurácia', score)

F1 Score (Macro) -  0.8388349789623075
Acurácia 0.8415492957746479
